In [3]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

import os 
import tensorflow as tf
from pathlib import Path
import cv2

# Run this before loading other dependencies, otherwise they might occupy memory on gpu 0 by default and it will stay that way

# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Or 2, 3, etc. other than 0

config = tf.compat.v1.ConfigProto(device_count={'GPU': 1}, allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 1.0
tf.compat.v1.Session(config=config)

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from src.architectures.simple.simple_base import SimpleBaseArchitecture
import numpy as np

from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from tensorflow.keras.optimizers import Adam, SGD
from src.architectures.benchmarks.benchmark_definitions import Chexpert_Benchmark, Chestxray14_Benchmark, simple_architecture_experiment, generate_benchmarks, METRICS, SINGLE_CLASS_METRICS, CHEXPERT_COLUMNS, CHESTXRAY14_COLUMNS
from src.metrics.metrics import F2Score
from src.metrics.losses import WeightedBinaryCrossentropy, BinaryCrossentropy, compute_class_weight
from src.utils.save_model import load_model

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:04:00.0, compute capability: 7.5



In [4]:
# color, affine + color, affine
model_ids = ["c55ed5c6-e879-11ea-9a58-0242ac110005", "2eb47be0-e88c-11ea-9a58-0242ac110005", "74d8220a-e6c2-11ea-b084-0242ac110005"]

In [5]:
paths = [load_model(id) for id in model_ids]

In [9]:
def build_model(architecture, num_classes, path):
    model = SimpleBaseArchitecture(architecture, num_classes, train_last_layer_only=False)
    model.load_weights(path)

    return model

In [10]:
models = [build_model(DenseNet121, 12, path) for path in paths]

In [11]:
from deepstack.base import KerasMember

member1 = KerasMember(name="model1", keras_model=models[0], train_batches=(x_train, y_train), val_batches=(x_val, y_val))
member2 = KerasMember(name="model2", keras_model=models[1], train_batches=(x_train, y_train), val_batches=(x_val, y_val))
member3 = KerasMember(name="model3", keras_model=models[2], train_batches=(x_train, y_train), val_batches=(x_val, y_val))
